In [66]:
import os
from PIL import Image
import numpy as np
import cPickle as pk
from sklearn.decomposition import PCA

In [2]:
# Function to recursively read images from location
def read_images(location, imageType = ("jpg")):
    
    imageFolders = []
    imageFiles = []
    try:
        fileList = os.listdir(location)
    except OSError:
        return imageFiles, imageFolders
        
    for fl in fileList:
        
        if fl.split(".")[-1] in imageType:
            imageFiles.append(location + "/" + fl)
            imageFolders.append(location.split("/")[-1])
        else:
            subFolder = location + "/" + fl
            folderSub, imageSub = read_images(subFolder)
#             imageFiles.append(imageSub)
#             imageFolders.append(folderSub)
            imageFiles = imageFiles + imageSub
            imageFolders = imageFolders + folderSub
    
    return imageFiles, imageFolders

In [3]:
LOCATION = 'faces94/'

In [4]:
iFiles, iFolders = read_images(LOCATION)

In [5]:
iFolders = np.array(iFolders)

In [6]:
allImages = np.empty([len(iFiles), 200, 180, 3], dtype="float32")

In [7]:
for i in range(len(iFiles)):
    if(i%500 == 0):
        print i
    im = Image.open(iFiles[i])
    allImages[i] = im
    # train_images = np.append(train_images, im)
    # train_labels = np.append(train_labels, iFolders[i])

0
500
1000
1500
2000
2500
3000


In [8]:
train_index = np.random.choice(a = range(len(allImages)), replace=False, size=100)

In [9]:
train_images = allImages[train_index,:,:]
test_index = np.setdiff1d(range(len(allImages)), train_index)
test_images = allImages[test_index,:,:]

In [10]:
train_labels = iFolders[train_index]
test_labels = iFolders[test_index]

In [11]:
# Conversion to greyscale
train_greyscale = np.mean(train_images, axis = 3)
test_greyscale = np.mean(test_images, axis = 3)

In [12]:
train_images = None
test_images = None
allImages = None

In [13]:
# Compute mean face
mean_face = np.mean(train_greyscale, axis=0)

In [14]:
mean_im = Image.fromarray(mean_face)
mean_im.show()

In [15]:
# Convert to 1D
train_image1d = train_greyscale.reshape([len(train_greyscale), 200*180])

In [16]:
train_image1d = train_image1d/255.

In [17]:
train_image1d.shape

(100, 36000)

In [22]:
eigen_pc = PCA(n_components = 10)

In [23]:
eigen_pc.fit(train_image1d)

PCA(copy=True, iterated_power='auto', n_components=10, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [24]:
X = eigen_pc.transform(train_image1d)

In [25]:
eigen_pc.components_.shape

(10, 36000)

In [26]:
X.shape

(100, 10)

In [27]:
np.unique(train_labels)

array(['9326871', '9332898', '9338462', '9338489', '9338497', '9338535',
       '9416994', 'admars', 'ahodki', 'ajsega', 'ambarw', 'anonym',
       'anpage', 'asewil', 'asheal', 'astefa', 'ccjame', 'cmkirk', 'cwang',
       'dagran', 'dcbowe', 'djbirc', 'djhugh', 'dmwest', 'drbost',
       'ekavaz', 'fordj', 'ggeorg', 'gjnorm', 'gmwate', 'gpapaz', 'gpsmit',
       'hartb', 'hensm', 'ieorf', 'jcarte', 'jdbenm', 'jgloma', 'jlemon',
       'jmedin', 'jrtobi', 'kaatki', 'kaknig', 'khughe', 'kjwith',
       'klclar', 'lfso', 'lyond', 'mbutle', 'mdpove', 'mefait', 'miaduc',
       'moors', 'mpetti', 'ndbank', 'ndhagu', 'nhrams', 'njmoor', 'npmitc',
       'nrclar', 'nrrbar', 'pmives', 'pspliu', 'ptnich', 'rgspru', 'robin',
       'rsanti', 'saedwa', 'sandm', 'sidick', 'smrobb', 'spacl', 'spletc',
       'svkriz', 'swewin', 'voudcx', 'whussa', 'wjalbe'], 
      dtype='|S7')

In [32]:
[i for i in range(len(test_labels)) if test_labels[i] == 'robin']

[2904,
 2905,
 2906,
 2907,
 2908,
 2909,
 2910,
 2911,
 2912,
 2913,
 2914,
 2915,
 2916,
 2917,
 2918,
 2919,
 2920,
 2921,
 2922]

In [68]:
test_img = test_greyscale[2912].reshape(1, 36000)/255.

In [82]:
test_labels[2912]

'robin'

In [83]:
im2 = Image.fromarray(test_greyscale[2912])

In [84]:
im2.show()

In [85]:
test_pc = eigen_pc.transform(test_img)

In [86]:
test_pc.shape

(1, 10)

In [87]:
sq_err = (X - test_pc) ** 2

In [88]:
sq_err.shape

(100, 10)

In [89]:
dist = np.sqrt(sq_err.sum(axis=1))

In [90]:
dist.shape

(100,)

In [91]:
np.argmin(dist)

32

In [92]:
train_labels[32]

'robin'

In [96]:
imt = Image.fromarray(train_greyscale[32])
imt.show()

In [97]:
im2.show()